In [1]:
import json, csv, torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM



In [1]:
import json, csv, torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "CohereForAI/aya-101"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

...

model-00010-of-00011.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(250112, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=4096, out_features=4096, bias=False)
              (k): Linear(in_features=4096, out_features=4096, bias=False)
              (v): Linear(in_features=4096, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 64)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=4096, out_features=10240, bias=False)
              (wi_1): Linear(in_features=4096, out_features=10240, bias=False)
   

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
json_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/cleaned_data/english_cleaned.json"
output_csv = "/content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_english.csv"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

all_qas = []

for article in data["data"]:
    for para in article["paragraphs"]:
        context = para["context"]
        for qa in para["qas"]:
            if not qa.get("answers"):
                continue
            all_qas.append({
                "id": qa["id"],
                "question": qa["question"],
                "context": context,
                "expected": qa["answers"][0]["text"]
            })

print(f"✅ Total English questions: {len(all_qas)}")


✅ Total English questions: 5928


In [4]:
BATCH_SIZE = 8  # Aya is heavy, keep conservative

with open(output_csv, "w", newline="", encoding="utf-8") as f_out:
    writer = csv.writer(f_out)
    writer.writerow(["id", "model_answer", "expected_answer"])

    num_batches = (len(all_qas) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_idx in tqdm(
        range(num_batches),
        desc="Generating English QA with Aya-101"
    ):
        torch.cuda.empty_cache()

        batch = all_qas[
            batch_idx * BATCH_SIZE : (batch_idx + 1) * BATCH_SIZE
        ]

        prompts = [
            f"Context: {e['context']}\nQuestion: {e['question']}\nAnswer:"
            for e in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )

        for i, entry in enumerate(batch):
            generated = decoded[i].strip().split("\n")[0]

            writer.writerow([
                entry["id"],
                generated,
                entry["expected"]
            ])

print("✅ Aya English CSV saved to:", output_csv)


Generating English QA with Aya-101: 100%|██████████| 741/741 [11:29<00:00,  1.07it/s]

✅ Aya English CSV saved to: /content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_english.csv


**HINDI**

In [5]:
json_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/cleaned_data/hindi_cleaned-filtered.json"
output_csv = "/content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_hindi.csv"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

all_qas = []

# Hindi is FLAT format
for item in data["data"]:
    if not isinstance(item, dict):
        continue

    answers = item.get("answers", {}).get("text", [])
    if not answers:
        continue

    all_qas.append({
        "id": item["id"],
        "question": item["question"],
        "context": item["context"],
        "expected": answers[0]
    })

print(f"✅ Total Hindi questions: {len(all_qas)}")


✅ Total Hindi questions: 5928


In [6]:
BATCH_SIZE = 8   # Aya is memory heavy — keep small

with open(output_csv, "w", newline="", encoding="utf-8") as f_out:
    writer = csv.writer(f_out)
    writer.writerow(["id", "model_answer", "expected_answer"])

    num_batches = (len(all_qas) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_idx in tqdm(
        range(num_batches),
        desc="Generating Hindi QA with Aya-101"
    ):
        torch.cuda.empty_cache()

        batch = all_qas[
            batch_idx * BATCH_SIZE : (batch_idx + 1) * BATCH_SIZE
        ]

        prompts = [
            f"Context: {e['context']}\nQuestion: {e['question']}\nAnswer:"
            for e in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )

        for i, entry in enumerate(batch):
            generated = decoded[i].strip().split("\n")[0]

            writer.writerow([
                entry["id"],
                generated,
                entry["expected"]
            ])

print("✅ Hindi Aya-101 CSV saved to:", output_csv)


Generating Hindi QA with Aya-101: 100%|██████████| 741/741 [20:05<00:00,  1.63s/it]

✅ Hindi Aya-101 CSV saved to: /content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_hindi.csv


KANNADA

In [7]:
json_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/cleaned_data/kannada_cleaned-filtered.json"
output_csv = "/content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_kannada.csv"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

all_qas = []

# Hindi is FLAT format
for item in data["data"]:
    if not isinstance(item, dict):
        continue

    answers = item.get("answers", {}).get("text", [])
    if not answers:
        continue

    all_qas.append({
        "id": item["id"],
        "question": item["question"],
        "context": item["context"],
        "expected": answers[0]
    })

print(f"✅ Total Kannada questions: {len(all_qas)}")


✅ Total Kannada questions: 5928


In [8]:
BATCH_SIZE = 8   # Aya is memory heavy — keep small

with open(output_csv, "w", newline="", encoding="utf-8") as f_out:
    writer = csv.writer(f_out)
    writer.writerow(["id", "model_answer", "expected_answer"])

    num_batches = (len(all_qas) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_idx in tqdm(
        range(num_batches),
        desc="Generating Hindi QA with Aya-101"
    ):
        torch.cuda.empty_cache()

        batch = all_qas[
            batch_idx * BATCH_SIZE : (batch_idx + 1) * BATCH_SIZE
        ]

        prompts = [
            f"Context: {e['context']}\nQuestion: {e['question']}\nAnswer:"
            for e in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )

        for i, entry in enumerate(batch):
            generated = decoded[i].strip().split("\n")[0]

            writer.writerow([
                entry["id"],
                generated,
                entry["expected"]
            ])

print("✅ Kannada Aya-101 CSV saved to:", output_csv)


Generating Hindi QA with Aya-101: 100%|██████████| 741/741 [19:37<00:00,  1.59s/it]

✅ Kannada Aya-101 CSV saved to: /content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_kannada.csv


**TAMIL**

In [10]:
json_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/cleaned_data/tamil_cleaned-filtered.json"
output_csv = "/content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_tamil.csv"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

all_qas = []

# Hindi is FLAT format
for item in data["data"]:
    if not isinstance(item, dict):
        continue

    answers = item.get("answers", {}).get("text", [])
    if not answers:
        continue

    all_qas.append({
        "id": item["id"],
        "question": item["question"],
        "context": item["context"],
        "expected": answers[0]
    })

print(f"✅ Total tamil questions: {len(all_qas)}")


✅ Total tamil questions: 5928


In [11]:
BATCH_SIZE = 8   # Aya is memory heavy — keep small

with open(output_csv, "w", newline="", encoding="utf-8") as f_out:
    writer = csv.writer(f_out)
    writer.writerow(["id", "model_answer", "expected_answer"])

    num_batches = (len(all_qas) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_idx in tqdm(
        range(num_batches),
        desc="Generating Tamil QA with Aya-101"
    ):
        torch.cuda.empty_cache()

        batch = all_qas[
            batch_idx * BATCH_SIZE : (batch_idx + 1) * BATCH_SIZE
        ]

        prompts = [
            f"Context: {e['context']}\nQuestion: {e['question']}\nAnswer:"
            for e in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )

        for i, entry in enumerate(batch):
            generated = decoded[i].strip().split("\n")[0]

            writer.writerow([
                entry["id"],
                generated,
                entry["expected"]
            ])

print("✅ Tamil Aya-101 CSV saved to:", output_csv)


Generating Tamil QA with Aya-101: 100%|██████████| 741/741 [17:19<00:00,  1.40s/it]

✅ Tamil Aya-101 CSV saved to: /content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_tamil.csv


MARATHI

In [14]:
json_path = "/content/drive/MyDrive/Project/Dataset/IndicSquad/cleaned_data/marathi_cleaned-filtered-1f.json"
output_csv = "/content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_marathi.csv"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

all_qas = []

# Marathi = FLAT format
for item in data["data"]["data"]:   # IMPORTANT: nested data
    if not isinstance(item, dict):
        continue

    answers = item.get("answers", {}).get("text", [])
    if not answers:
        continue

    all_qas.append({
        "id": item["id"],
        "question": item["question"],
        "context": item["context"],
        "expected": answers[0]
    })

print(f"✅ Total Marathi questions: {len(all_qas)}")


✅ Total Marathi questions: 5928


In [15]:
BATCH_SIZE = 8

with open(output_csv, "w", newline="", encoding="utf-8") as f_out:
    writer = csv.writer(f_out)
    writer.writerow(["id", "model_answer", "expected_answer"])

    num_batches = (len(all_qas) + BATCH_SIZE - 1) // BATCH_SIZE

    for batch_idx in tqdm(
        range(num_batches),
        desc="Generating Marathi QA with Aya-101"
    ):
        torch.cuda.empty_cache()

        batch = all_qas[
            batch_idx * BATCH_SIZE : (batch_idx + 1) * BATCH_SIZE
        ]

        prompts = [
            f"Context: {e['context']}\nQuestion: {e['question']}\nAnswer:"
            for e in batch
        ]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=64,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )

        for i, entry in enumerate(batch):
            generated = decoded[i].strip().split("\n")[0]

            writer.writerow([
                entry["id"],
                generated,
                entry["expected"]
            ])

print("✅ Marathi Aya-101 CSV saved to:", output_csv)


Generating Marathi QA with Aya-101: 100%|██████████| 741/741 [19:55<00:00,  1.61s/it]

✅ Marathi Aya-101 CSV saved to: /content/drive/MyDrive/Project/Dataset/IndicSquad/OUTPUTS/AYA/aya_marathi.csv
